In [1]:
import parse_tfl_json
import google

In [2]:
import pandas as pd 
from mylibrary.connections import cursor, conn, engine, Automapped_Base, session

sql = """
select * from tt_h.stations_journeytime 
where tfl_message = 'ok' 
limit 100
"""
df = pd.read_sql(sql, conn)

/Users/robinlinacre/anaconda/lib/python3.6/site-packages/sqlalchemy/dialects/postgresql/base.py:2526: SAWarning: Did not recognize type 'geometry' of column 'geom'
  (attype, name))


In [3]:
from process_journeys import get_london_icscodes, add_cycle_and_total_time, remove_journeys_not_arriving_clondon
from google import get_cycle_info
import parse_tfl_json
import json

In [4]:
# df = df.loc[8:9,:]

In [5]:
for r in df.iterrows():
    index = r[0]
    row = r[1]

    try:
        j1 = row["tfl_response"]
        j2 = json.loads(j1)
    except:
        print("parse problem")
        continue
    
    try:
        journeys = j2["journeys"]
    except:
        print("no journeys in json")
        continue
    
    
    for journey in journeys:
        legs = journey["legs"]
        journey["legs"] = parse_tfl_json.remove_non_national_rail(legs)
    

    
    # Keep only journeys with a national rail component
    journeys = [j for j in journeys if len(j["legs"]) > 0]
    
    
    
    # Keep only journeys that end in central London
    
    journeys = remove_journeys_not_arriving_clondon(journeys)

    
    if len(journeys) == 0:
        continue

    journeys = add_cycle_and_total_time(journeys, row["tfl_dest"])

    journeys = sorted(journeys, key = lambda x: x["total_time"])

    journey = journeys[0]
    legs = journey["legs"]

    summary = parse_tfl_json.summarise_legs(legs)
    df.loc[index, "natrail_journey_summary"] = summary

    df.loc[index, "natrail_train_changes"] = parse_tfl_json.num_changes(legs)
    df.loc[index, "final_arrival"] = parse_tfl_json.final_arrival(legs)

    travel_time_minutes = parse_tfl_json.get_total_travel_time(legs) 
    df.loc[index, "natrail_journey_minutes"] = travel_time_minutes

    lat_lng = parse_tfl_json.lat_lng_from_legs(legs)

    df.loc[index, "depart_lat"] = lat_lng["depart"]["lat"]
    df.loc[index, "depart_lng"] = lat_lng["depart"]["lng"]
    df.loc[index, "arrive_lat"] = lat_lng["arrive"]["lat"]
    df.loc[index, "arrive_lng"] = lat_lng["arrive"]["lng"]

    df.loc[index, "cycle_minutes"] = journey["cycle_minutes"]
    df.loc[index, "cycle_miles"] = journey["cycle_miles"]
    df.loc[index, "total_journeytime"] = journey["cycle_minutes"] + travel_time_minutes

    
    
cols = [c for c in df.columns if c not in ["id", "tfl_request", "tfl_response", "querydict"]]
df2 = df[cols]

In [6]:
df2

,nlc,icscode,station_name,tfl_message,tfl_dest,natrail_journey_summary,natrail_train_changes,final_arrival,natrail_journey_minutes,depart_lat,depart_lng,arrive_lat,arrive_lng,cycle_minutes,cycle_miles,total_journeytime
0,9657,90005856,Paisley Canal,ok,SW1H9AJ,Paisley Canal Rail Station → Glasgow Central R...,1.0,Euston Station,305.0,55.840076,-4.424117,51.528952,-0.135000,17.235056,4.021513,322.235056
1,4876,90001586,Colwall,ok,SW1H9AJ,Colwall Rail Station → Birmingham New Street R...,1.0,Euston Station,170.0,52.079867,-2.356950,51.528952,-0.135000,17.235056,4.021513,187.235056
2,5139,1001124,Grove Park,ok,SW1H9AJ,Grove Park Station → London Bridge Station,0.0,London Bridge Station,19.0,51.430783,0.022082,51.505133,-0.085439,13.333734,3.111205,32.333734
3,4876,90001586,Colwall,ok,SW1H9AJ,Colwall Rail Station → Birmingham New Street R...,1.0,Euston Station,170.0,52.079867,-2.356950,51.528952,-0.135000,17.235056,4.021513,187.235056
4,5139,1001124,Grove Park,ok,SW1H9AJ,Grove Park Station → London Bridge Station,0.0,London Bridge Station,16.0,51.430783,0.022082,51.505133,-0.085439,13.333734,3.111205,29.333734
5,5006,90001027,Broadstairs,ok,SW1H9AJ,Broadstairs Rail Station → St Pancras Internat...,0.0,St Pancras International Station,84.0,51.360673,1.433561,51.532313,-0.126400,18.859498,4.400549,102.859498
6,3776,90002610,Eskbank,ok,SW1H9AJ,Eskbank Rail Station → Edinburgh Rail Station ...,1.0,London King's Cross Rail Station,285.0,55.881801,-3.083073,51.531579,-0.123243,19.895412,4.642263,304.895412
7,9657,90005856,Paisley Canal,ok,SW1H9AJ,Paisley Canal Rail Station → Glasgow Central R...,1.0,Euston Station,305.0,55.840076,-4.424117,51.528952,-0.135000,17.235056,4.021513,322.235056
8,7454,1001235,Rainham (Essex),ok,SW1H9AJ,Rainham Station → Limehouse Station,0.0,Limehouse Station,21.0,51.516823,0.190465,51.512657,-0.039596,20.787523,4.850422,41.787523
9,7454,1001235,Rainham (Essex),ok,SW1H9AJ,Rainham Station → Limehouse Station,0.0,Limehouse Station,21.0,51.516823,0.190465,51.512657,-0.039596,20.787523,4.850422,41.787523


In [ ]:
df3 = df2.sort_values(["station_name", "total_journeytime"]).drop_duplicates("station_name")

In [ ]:
df_arrivals = df2.drop_duplicates("final_arrival")[["final_arrival", "arrive_lat", "arrive_lng"]]


In [ ]:
df_arrivals = df_arrivals[["final_arrival", "cycle_miles_london", "cycle_minutes_london"]]
df4 = df3.merge(df_arrivals, how = "left", left_on = "final_arrival", right_on = "final_arrival")

In [ ]:
keep = ["station_name", "natrail_journey_summary", "depart_lat", "depart_lng", "natrail_journey_minutes","cycle_minutes_london"]
df4 = df4[keep]
df4["total_minutes"] = df4["cycle_minutes_london"] + df4["natrail_journey_minutes"]


In [ ]:
# Want to update all the names to be cw specific 
df5 = df4.rename(columns={"depart_lat" : "lat", "depart_lng": "lng", "station_name": "name"})

In [ ]:
df5.to_csv("cw_to_map.csv", encoding="utf-8", index=False)

In [ ]:
df5.columns

In [ ]:
# To do - pick best journey time by total, not just the time on the train.  
# We can filter out suggested journeys where the lat lng is outside of central London

In [ ]:
journeys[1]